**MDS7203 Modelos Generativos Profundos, Primavera 2023**

# Laboratorio 2: Modelos Autoregresivos

**Profesor**: Felipe Tobar, **Auxiliares**: Cristóbal Alcázar, Camilo Carvajal Reyes, **Ayudante**: Joaquín Barceló.

**Fecha de entrega**: viernes 29 de septiembre 2023

**Nombre: COLOCAR AQUÍ SU NOMBRE**

**Instrucciones**: El presente notebook contiene enunciado e instrucciones para la realización del laboratorio. Usted deberá completar los códigos (en este archivo o en una copia del mismo) donde se le pida hacerlo. Usted deberá entregar el notebook con sus respuestas, cumpliendo lo siguiente:

- Los comentarios en código deben ser concisos pero claros. No se evaluarán sub-preguntas donde solo exista código sin comentarios pertinentes.
- El código debe ser ordenado y ejectuable. No se evaluarán notebooks o scripts que generen errores en su ejecución. Se aconseja resetear la kernel y corroborar la correcta execución de todas las celdas antes de ejecutar el entrenamiento de su modelo.
- Si bien se aconseja el uso de internet y otras herramientas para asistir su trabajo, asi como discusiones con el ED y estudiantes, el código que entregue debe ser de su autoría.

Algunos links útiles:

* [Language Models are Unsupervised Multitask Learners](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf)
* [GPT2, original blog post](https://openai.com/research/better-language-models)


### Resumen de preguntas

- [ ] a) (0,5 ptos.) Definición de diccionarios para vocabulario

- [ ] b) (bonus) Utilización de embeddings previo a la normalización

- [ ] c) (bonus) Escalamiento por $1/\sqrt{d_k}$.

- [ ] d) (1.5 ptos.) Creación de clase `Head`.

- [ ] e) (1 pto.) Implementación de clase `FeedForward`.

- [ ] f) (0.5 ptos.) Relación entre hiper-parámetros n_head y head_size.

- [ ] g) (0.5 ptos.) Forward pass en `EncoderBlock`.

- [ ] h) (1 pto.) Implementación clase `GPTLM`.

- [ ] g) (1 pto.) Training loop.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


## 1. Corpus 📖

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-04 21:58:34--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.005s  

2023-09-04 21:58:50 (205 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

print(f"Tamaño del corpus: {len(text):,} caracteres")

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


> a) (0.5 ptos.) Dada una lista de ordenada de caracteres, defina:
> - stoi: un diccionario caracter -> índice
> - itos: un diccionario índice -> caracter
> Con lo anterior, defina dos funciones encode y decode que tomen un string y una lista de índices respectivamente y devuelvan una lista de índices y un string según corresponda.

In [ ]:
# --------------------------------------------

stoi = pass
itos = pass

# encoder: toma un string, devuelve una lista de índices
encode = lambda s: None  # CAMBIAR

# decoder: toma una lista de índices, devuelve un string
decode = lambda l: None  # CAMBIAR

# --------------------------------------------

In [ ]:
assert encode('hola, que tal?') == [46, 53, 50, 39, 6, 1, 55, 59, 43, 1, 58, 39, 50, 12], 'Verifica que el output entregue una lista de enteros'
assert decode(encode('hola, que tal?')) == 'hola, que tal?', 'Debe ser un string'

Nuestro modelo no entiende el lenguaje directamente, sino que los representa como números. Pasamos el corpus completo a su representación de enteros, usando el `stoi` (aka _string-to-index_).

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
data.shape

torch.Size([1115394])

In [ ]:
N=100
print(f"Texto con los primeros {N} caracteres:\n----------------------------------------------\n")
print(text[:N])
print("\n----------------------------------------------\nSu representación como tensor de PyTorch...\n")
print(data[:N])

Texto con los primeros 100 caracteres:
----------------------------------------------

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

----------------------------------------------
Su representación como tensor de PyTorch...

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


## 2. Separar el dataset 🔨 y 🎓

In [ ]:
n = int(0.9 * len(data))  # 90%
train_data = data[:n]
val_data = data[n:]
print(f"--> Tamaño del corpus de entrenamiento: {len(train_data):,} ({(train_data.shape[0] / data.shape[0]):.2f}) caracteres")
print(f"--> Tamaño del corpus de validación: {len(val_data):,} ({(val_data.shape[0] / data.shape[0]):.2f}) caracteres")

--> Tamaño del corpus de entrenamiento: 1,003,854 (0.90) caracteres
--> Tamaño del corpus de validación: 111,540 (0.10) caracteres


Sobre la mécanica de datos y etiquetas,

* Accedemos a los datos a partir de "fragmentos contextuales"; esto es un bloque de texto en representación númerica de tamaño `block_size`
* El modelo es semi-supervisado, es decir, búscamos entrenar un modelo de tal forma que dado ${x}_{i:j}$ _tokens_, vamos a predecir el siguiente _token_ $x_{j+1}$
* Las etiquetas emergen del mismo bloque contextual moviendo la ventana con un _offset_ de 1.

Por ejemplo, dado un bloque de tamaño 8,


In [ ]:
block_size = 13
print(f"Una bloque contextual (X, Y) será:\n")
print(f"X: {[x.item() for x in data[:block_size]]}")
print(f"  --> decode(X): {decode([x.item() for x in data[:block_size]])}")
print('------------------------------------')
print(f"Y: {[x.item() for x in data[1:block_size+1]]}")
print(f"  --> decode(Y): {decode([y.item() for y in data[1:block_size+1]])}")

Una bloque contextual (X, Y) será:

X: [18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52]
  --> decode(X): First Citizen
------------------------------------
Y: [47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10]
  --> decode(Y): irst Citizen:


Sin embago, dentro de cada bloque contextual ocupamos la información de manera autoregresiva, generando múltiple observaciones a partir de este...

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Cuando el input es {context} el target es: {target}")

Cuando el input es tensor([18]) el target es: 47
Cuando el input es tensor([18, 47]) el target es: 56
Cuando el input es tensor([18, 47, 56]) el target es: 57
Cuando el input es tensor([18, 47, 56, 57]) el target es: 58
Cuando el input es tensor([18, 47, 56, 57, 58]) el target es: 1
Cuando el input es tensor([18, 47, 56, 57, 58,  1]) el target es: 15
Cuando el input es tensor([18, 47, 56, 57, 58,  1, 15]) el target es: 47
Cuando el input es tensor([18, 47, 56, 57, 58,  1, 15, 47]) el target es: 58
Cuando el input es tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) el target es: 47
Cuando el input es tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) el target es: 64
Cuando el input es tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) el target es: 43
Cuando el input es tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) el target es: 52
Cuando el input es tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]) el target es: 10


Por lo tanto, cada bloque contextual, genera un número de observaciones igual a su tamaño.

En términos de _batches_, podemos procesar en paralelo, múltiples bloques contextuales. Lo importante es que cada bloque contextual es independiente, y no hay computo que ocurra a nivel transversal, sino paralelo entre estos. No se mezclan las secuencias autoregresivas de cada contexto.

In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"Cuando el input es {context.tolist()} el target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
Cuando el input es [24] el target: 43
Cuando el input es [24, 43] el target: 58
Cuando el input es [24, 43, 58] el target: 5
Cuando el input es [24, 43, 58, 5] el target: 57
Cuando el input es [24, 43, 58, 5, 57] el target: 1
Cuando el input es [24, 43, 58, 5, 57, 1] el target: 46
Cuando el input es [24, 43, 58, 5, 57, 1, 46] el target: 43
Cuando el input es [24, 43, 58, 5, 57, 1, 46, 43] el target: 39
Cuando el input es [44] el target: 53
Cuando el input es [44, 53] el target: 56
Cuando el input es [44, 53, 56] el target: 1
Cuando el input es [44, 53, 56, 1] el target: 58
Cuando el input es [44

Lo que recibirá la red como _input_ será:

In [ ]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

## 3. Baseline

Creamos un modelo base clásico, para luego compararlo con nuestro Transformer.

In [ ]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # cada token lee sucesivamente los logits para el token siguiente de una lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx y targets son ambos tensores de tamaño (B,T) con elementos enteros
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx es un arreglo (B, T) de indices del contexto actual
        for _ in range(max_new_tokens):
            # obtener predicciones
            logits, loss = self(idx)
            # concentrarse en el último paso
            logits = logits[:, -1, :]  # se convierte en (B, C)
            # aplicamos softmax para obtener probabilidades
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # samplear de la distribución
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # agregar el índice de la muestra a la secuencia actual
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


## 4. Self-attention

En la sección anterior, vimos que cada _token_ toma una representación vectorial llamada _embeddings_.

Nuestro corpus contiene $65$ caracteres únicos con un _embedding_ asociado a cada _token_. La idea de representar el lenguaje en términos de tokens, y estos a su vez en vectores, es que podemos aprender estas representaciones vectoriales a partir de los datos. Sin embargo, la representación es única, y muchas veces un mismo _token_ puede tener distintos significados según su contexto. [INSERTAR EJEMPLO].

La idea principal de _self-attention_ es utilizar la secuencia de _embeddings_ dentro de un contexto para computar un promedio ponderado a partir de estos. Dado una secuencia de _embeddings_ de _tokens_ $x_1, \dots, x_n$, el mecanismo de _self-attention_ (o auto-atención) produce una nueva secuencia de _embeddings_ $x'_1, \dots, x'_n$, donde cada $x'_i$ es una combinación lineal de todos los $x_j$:

$$
x'_i = \sum_{j=1}^{n} \alpha_{ij} x_{j}
$$

Los coeficientes $\alpha_{ij}$ se llaman ponderadores de atención y están normalizados tal que $\sum_{j}\alpha_{ji}=1$.

En términos sencillos, construiremos un mecanismo de comunicación entre distintos tokens dentro del bloque de contexto, que se representará por una colección de ponderadores en una matriz. Esta colección de ponderadores la llamaremos matriz de atención (o self-attention) y nos permitirá vía la operación de multiplicación de matrices, agregar distintos valores dentro de un bloque contextual en una sola cantidad. Spoiler, estos pesos serán data-dependientes.

Comencemos emulando la operación con pesos fijos, usaremos la parte triangular inferior de una matriz identidad de 3x3, la cual normalizaremos a nivel de fila.

In [ ]:
# Ejemplo de juguete que ilustra como la multiplicación matricial puede ser usada para una adición con pesos
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print('a=',a,'\n')
print('b=',b,'\n')
print('c=',c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


Notemos que $c$ tiene en cada fila los resultados de los valores acumulados de $b$ según los ponderadores de $a$.

Pregunta: si $b$ es un tensor de $3\times 2$, que representa 2 bloques contextuales de 3 tokens (piense en los contextos `yo quiero manjar` y `mañana es lunes`). Cómo se interpreta que $a$ sea un tensor de $3
\times 3$ triangular inferior dentro de un modelo autoregresivo?


Vamos a crear un _batch_ con datos síntetico de tamaño `B`, donde cada bloque contextual será de largo $T$, y cada _token_ que compone el contexto se representa por $C$ dimensiones (i.e. tamaño del _embedding_).

In [ ]:
torch.manual_seed(1337)

B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# Versión usando softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

Los ponderadores anteriores son uniformes, ahora introducimos los conceptos de _queries_ y _keys_ para ver como computar los ponderadores a partir de los datos. ALINEAR EXPLICACION CON DIAGRAMA DE RASCHKA.

* Un _query_ $q(\cdot)$ corresponde a una proyección lineal de la representación de _embeddings_ de un token particular. Por ejemplo, se proyecto $\mathbb{R}^{C}\rightarrow \mathbb{R}^{H}$.
* Los _keys_ es la matriz $K\in\mathbb{R}^{T\times H}$ que contiene proyecciones lineales de todos los _embeddings_ de tokens dentro del contexto, incluído el token que es el query. La proyección lineal de los _keys_ es de igual tamaño (i.e. $H$) que la proyección del _query_.
* Los ponderadores para cada _query_ se obtiene a partir de qué tan próximo se relaciona un token respecto al resto de los token dentro de un contexto. Por ejemplo, $q(x_i) \times K$

In [ ]:
from IPython.display import Image
Image("https://sebastianraschka.com/images/blog/2023/self-attention-from-scratch/attention-scores.png")

<img src="image-20230904-194635.png" width="" align="" />

> b) (Bonus): Explique porqué no se utilizan directamente los _embeddings_ para computar la matriz de atención previo a la normalización, i.e. `(T,B).dot((B,T))`, en vez de usar las proyecciones $QK^\top$.

> c) (Bonus): Explique los argumentos detras de escalar por $1/\sqrt{d_k}$ referidos en el paper _[Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)_ (Vaswani 2017).

In [ ]:
# colocar seed como su RUT
torch.manual_seed(1337)

B,T,C = 4,8,32  # batch, time, channels
x = torch.randn(B,T,C)

torch.Size([4, 8, 16])

Ejemplo de aplicación de módulo de atención

In [ ]:
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x)  # (B, T, 16)
wei =  q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x

out.shape

Observaciones:
- Atención es un **mecanismo de comunicación**. Puede ser entendido como nodos en un grafo dirigido conectándose unos con otros y agregando información con una suma ponderada de todos los nodos que apuntan a ellos, con pesos dependientes de los datos.
- No hay una noción de espacio. Atención simplemente actua sobre el conjunto de vectores. Es por este motivo que se necesitan encoders posicionales.
- Cada punto dentro de un batch es, desde luego, procesado de manera independiente y nunca intractua con los otros.
- En un bloque de atención "encoder" basta comentar la linea que hace masking con `tril`, que hace que los tokens se comuniquen todos con todos. El bloque anterior se llama "decoder" porque aplica un masking triangular y se encuentre frecuentemente en configuraciones autoregresivas.
- "auto-atención" (_self-attention_) sólo signfica que tanto _keys_ como _values_ son producidas desde la misma fuente que las _queries_. En "atención-cruzada" (_cross-attention_), las _queries_ vienen de $x$, pero _keys_ y _values_ vienen de otra fuente externa (como puede ser un modulo encoder).
- Atención "escalada" divide `wei` por $\frac{1}{\sqrt{head\_size}}$. Esto hace que cuando los input $Q$ y $K$ tengan varianza unitaria, `wei` también tendrá varianza unitaria y evitará la saturación de la Softmax.

> d) (1.5 ptos.) Cree una clase `Head`que implemente un módulo de auto-atención.

In [ ]:
class Head(nn.Module):
    """ Una cabeza de auto-atención """

    def __init__(self, head_size):
        super().__init__()
        # ------------------------
        self.key = None  # CAMBIAR

        # etc ...

        # ------------------------
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input size (batch, time-step, channels)
        # output size (batch, time-step, head size)
        B,T,C = x.shape
        k = pass   # (B,T,hs)
        q = pass   # (B,T,hs)
        # --------------------------------------

        # Computar los score de atención ("affinities")

        # --------------------------------------
        wei = pass # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = pass # (B, T, T)
        wei = pass # (B, T, T)
        wei = self.dropout(wei)
        # --------------------------------------

        # Adición (con pesos) de las atenciones

        # --------------------------------------

        v = self.value(x)  # (B,T,hs)
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        
        return out

La arquitectura decoder del paper Transformer implementa varias versiones de _self-attention_ en paralelo, cada una es una "cábeza de atención", y estas concatenan sus resultados en un modulo conocido como `MultiHeadAttention`.

In [ ]:
class MultiHeadAttention(nn.Module):
    """ Múltiples cabezas de auto-atención en paralelo """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

> e) (1 pto.) Implemente una clase `FeedForward` como se describe en el artículo ["Attention is all you need, Vaswani et al."](https://arxiv.org/pdf/1706.03762.pdf).

In [ ]:
class FeedFoward(nn.Module):
    """ 
        Implementar FeedForward descrita en sección:
         "3.3 Position-wise Feed-Fordward Networks", paper
         "Attention is All You Need"
        https://arxiv.org/pdf/1706.03762.pdf
        
        in: n_embd
        out: n_embd
    """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            #nn.Linear(n_embd, 4 * n_embd),
            #nn.ReLU(),
            #nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
Image("http://jalammar.github.io/images/gpt2/gpt2-transformer-block-vectors-2.png")

<img src="image-20230904-203151.png" width="" align="" />

> f) (0.5 ptos.) Explique la relación entre los hiperparámetros `n_head` y `head_size` de la clase `MultiHeadAttention`. Piense en su rol dentro del bloque decoder (i.e. atención + feedforward).

> g) (0.5 ptos.) Complete el paso _forward_ de la clase `EncoderBlock`. Recuerde en particula incorporar las conexiones residuales (_skip connections_).

In [ ]:
class EncoderBlock(nn.Module):
    """  BloqueTransformer: COMUNICACIÓN seguida de CÓMPUTO """

    def __init__(self, n_embd, n_head):
        # n_embd: dimensión de embeddings, n_head: número de cabezas de atención
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # ---------------------

        x = pass  # completar

        # ----------------------

        return x

## 5. Modelo GPT: Juntando todo

> h) (1 pto.) Complete el código de la clase `GPTLanguageModel`procesando adecuadamente el input del modelo. Complete además el método `generate` para samplear elementos que completen auto-regresivamente una secuencia.

In [ ]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # cada token lee directamente los logits para el token siguiente de una lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx y targets son ambos tensores (B,T) de enteros
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)

        # -----------------------------

        x = pass  # DEFINIR x, con shape (B,T,C)

        # procesar x...

        logits = pass  # CALCULAR logits (B,T,vocab_size)

        # -------------------------------

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx es un arreglo (B, T) de índices en el contexto actual
        for _ in range(max_new_tokens):
            # restringir idx a los últimos block_size tokens
            idx_cond = idx[:, -block_size:]
            # obtener las predicciones
            logits, loss = self(idx_cond)
            # enfocarse sólo en el último paso
            logits = logits[:, -1, :]  # se convierte en size (B, C)
            # ------------------------------------------------------------

            # aplicar softmax para obtener las probabilidades
            probs = pass  # tensor de dimensionalidad (B, C)

            # definir una distribución apropiada para samplear usando las probabilidades
            idx_next = pass  # tensor de índices de dim (B, 1)

            # adjuntar el token generado a la secuencia (actualizando idx)
            idx = pass  # tensor resultante de dimensionalidad (B, T+1)
            
            # ------------------------------------------------------------

        return idx


In [ ]:
model = GPTLanguageModel()
m = model.to(device)
# printear el número de parámetros del modelo
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# definir el optimizador de PyTorch
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

> g) (1 pto.) Complete el bucle de entrenamiento usando comandos de `pytorch.optimize` conocidos. Ejecute el entrenamiento (se recomienda dejarlo corriendo e ir hacer algo más...). Corrobore que se modelo es capaz de generar texto.

In [ ]:
# hiper-parámetros
batch_size = 64
block_size = 256  # largo de ventana máximo para considerar en la precisión

max_iters = 5000
eval_interval = 500

learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

# colocar RUT como semilla
torch.manual_seed(1337)

for iter in range(max_iters):

    # de vez en cuando evaluar la loss en los conjuntos de train y evaluación
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # samplear un batch de datos
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)

    # ----------------------------------------

    # activar el optimizador y hacer el paso backward con el resultado de la función loss

    # ----------------------------------------

## 6. Generando secuencias de texto con el modelo

In [ ]:
# Generar usando el modelo
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

# Para escribir en un archivo
# open('output.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=42603f8c-d572-4f30-b66f-2f8f2cb33526' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>